# EN3160 Assignment 3 on Neural Networks

Instructed by Dr. Ranga Rodrigo

Done by Jayakumar W.S. (210236P)

### Introduction

This assignment is focused on implementing neural networks for image classification. This is done by using:
1. Our own neural network implementation
2. An implementation of LeNet-5
3. An implementation of ResNet-18

### Import necessary libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

### Dataloading

In [2]:
transform = transforms.Compose ([ transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5) , (0.5, 0.5, 0.5))])
batch_size = 32
trainset = torchvision.datasets.CIFAR10(root= './data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root= './data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device begin used : {device}")

100%|██████████| 170498071/170498071 [13:45<00:00, 206435.00it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Device begin used : cuda


### Our own architecture

#### Define Network Parameters

In [3]:
Din = 3*32*32 # Input size (flattened CIFAR=10 image size)
K = 10 # Output size (number of classes in CIFAR=10)
std = 1e-5
# Initialize weights and biases
w = torch.randn(Din, K, device=device, dtype=torch.float, requires_grad=True) * std
b = torch.randn(K, device=device, dtype=torch.float, requires_grad=True)
# Hyperparameters
iterations = 20
lr = 2e-6 # Learning rate
lr_decay = 0.9 # Learning rate decay
reg = 0 # Regularization
loss_history = [ ]

In [4]:
for t in range(iterations):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get inputs and labels
        inputs, labels = data
        Ntr = inputs.shape[0]  # Batch size
        x_train = inputs.view(Ntr, -1).to(device)  # Flatten input to (Ntr, Din)
        y_train_onehot = nn.functional.one_hot(labels, K).float().to(device)  # Convert labels to one-hot

        # Forward pass
        y_pred = x_train.mm(w) + b  # Output layer activation

        # Loss calculation (Mean Squared Error with regularization)
        loss = (1/Ntr) * torch.sum((y_pred - y_train_onehot) ** 2) + reg * torch.sum(w ** 2)
        loss_history.append(loss.item())
        running_loss += loss.item()

        # Backpropagation
        dy_pred = (2.0 / Ntr) * (y_pred - y_train_onehot)
        dw = x_train.t().mm(dy_pred) + reg * w
        db = dy_pred.sum(dim=0)

        # Parameter update
        w = w - lr * dw
        b = b - lr * db

    print(f"Epoch {t + 1} / {iterations}, Loss: {running_loss / len(trainloader)}")

    # Learning rate decay
    lr *= lr_decay

Epoch 1 / 20, Loss: 12.58823975827247
Epoch 2 / 20, Loss: 11.040970125033622
Epoch 3 / 20, Loss: 10.36810743206217
Epoch 4 / 20, Loss: 9.981834405824609
Epoch 5 / 20, Loss: 9.722565463621953
Epoch 6 / 20, Loss: 9.529328597331764
Epoch 7 / 20, Loss: 9.375695554095053
Epoch 8 / 20, Loss: 9.253262897828261
Epoch 9 / 20, Loss: 9.14980258319291
Epoch 10 / 20, Loss: 9.063382420567313
Epoch 11 / 20, Loss: 8.986990968004985
Epoch 12 / 20, Loss: 8.922773878618608
Epoch 13 / 20, Loss: 8.866755202727218
Epoch 14 / 20, Loss: 8.818267561118724
Epoch 15 / 20, Loss: 8.776347569754241
Epoch 16 / 20, Loss: 8.737724349960942
Epoch 17 / 20, Loss: 8.705339573624038
Epoch 18 / 20, Loss: 8.676764998005813
Epoch 19 / 20, Loss: 8.650618852748371
Epoch 20 / 20, Loss: 8.627122077282925


In [3]:
Din = 3*32*32 # Input size (flattened CIFAR=10 image size)
K = 10 # Output size (number of classes in CIFAR=10)
std = 1e-5
# Initialize weights and biases
w1 = torch.randn(Din, 100, device=device, requires_grad=True)
b1 = torch.zeros(100, device=device, requires_grad=True)
w2 = torch.randn(100, K, device=device, requires_grad=True)
b2 = torch.zeros(K, device=device, requires_grad=True)
# Hyperparameters
iterations = 20
lr = 2e-6 # Learning rate
lr_decay = 0.9 # Learning rate decay
reg = 0 # Regularization
loss_history = [ ]

#### Training loop

In [5]:
for t in range(iterations) :
    running_loss = 0.0
    for i , data in enumerate(trainloader, 0) :
        # Get inputs and labe l s
        inputs , labels = data
        Ntr = inputs.shape[0] # Batch size
        x_train = inputs.view(Ntr, -1).to(device) # Flatten input to (Ntr, Din)
        y_train_onehot = nn.functional.one_hot(labels, K).float().to(device) # Convert labe l s to one=hot # Forward pass
        hidden = x_train.mm(w1) + b1
        y_pred = hidden.mm(w2) + b2
        # Loss calculation (Mean Squared Error with regularization)
        loss = (1/Ntr) * torch.sum((y_pred - y_train_onehot) ** 2) + reg * (torch.sum(w1 ** 2) + torch.sum(w2 ** 2))
        loss_history.append(loss.item())
        running_loss += loss.item()
        # Backpropagation
        dy_pred = (2.0 / Ntr) * (y_pred - y_train_onehot)
        dhidden = dy_pred.mm(w2.t()) 
        dw2 = hidden.t().mm(dy_pred) + reg * w2
        db2 = dy_pred.sum(dim=0)
        dw1 = x_train.t().mm(dhidden) + reg * w1
        db1 = dhidden.sum(dim=0)
        # Parameter update
        w2 = w2 - lr * dw2
        b2 = b2 - lr * db2
        w1 = w1 - lr * dw1
        b1 = b1 - lr * db1
    print(f"Epoch {t+1} / {iterations} , Loss : {running_loss/len(trainloader)}")
    # Learning rat e decay
    lr *= lr_decay

Epoch 1 / 20 , Loss : 62761.097278245965
Epoch 2 / 20 , Loss : 12057.843131760337
Epoch 3 / 20 , Loss : 5480.4051508204875
Epoch 4 / 20 , Loss : 3028.8498120445206
Epoch 5 / 20 , Loss : 1866.6540542182752
Epoch 6 / 20 , Loss : 1239.1462293394238
Epoch 7 / 20 , Loss : 870.2966193981988
Epoch 8 / 20 , Loss : 639.2929228271953
Epoch 9 / 20 , Loss : 487.3385212044646
Epoch 10 / 20 , Loss : 383.4728423720816
Epoch 11 / 20 , Loss : 309.9592100644981
Epoch 12 / 20 , Loss : 256.5315048598511
Epoch 13 / 20 , Loss : 216.70926232316597
Epoch 14 / 20 , Loss : 186.42874668777867
Epoch 15 / 20 , Loss : 162.99634342596306
Epoch 16 / 20 , Loss : 144.50509007039622
Epoch 17 / 20 , Loss : 129.7724288188717
Epoch 18 / 20 , Loss : 117.8400370475198
Epoch 19 / 20 , Loss : 108.07219468578649
Epoch 20 / 20 , Loss : 100.00710443068374


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self, Din, H, Dout):
        super(NeuralNetwork, self).__init__()
        self.linear1 = nn.Linear(Din, H)
        self.linear2 = nn.Linear(H, Dout)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [5]:
model = NeuralNetwork(Din, 100, K).to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=reg)

In [6]:
for t in range(iterations):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get inputs and labels
        inputs, labels = data
        Ntr = inputs.shape[0]  # Batch size
        x_train = inputs.view(Ntr, -1).to(device)  # Flatten input to (Ntr, Din)
        y_train = labels.to(device)  # Convert labels to one-hot

        # Forward pass
        y_pred = model(x_train)

        # Loss calculation
        loss_val = loss(y_pred, y_train)
        loss_history.append(loss_val.item())
        running_loss += loss_val.item()

        # Backpropagation
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

    print(f"Epoch {t + 1} / {iterations}, Loss: {running_loss / len(trainloader)}")

Epoch 1 / 20, Loss: 2.1908527879629545
Epoch 2 / 20, Loss: 2.059300367220502
Epoch 3 / 20, Loss: 1.9850221211835504
Epoch 4 / 20, Loss: 1.9329886095537563
Epoch 5 / 20, Loss: 1.8936424514687527
Epoch 6 / 20, Loss: 1.8625754042458855
Epoch 7 / 20, Loss: 1.8372612156626968
Epoch 8 / 20, Loss: 1.8161037901007664
Epoch 9 / 20, Loss: 1.797995209312561
Epoch 10 / 20, Loss: 1.782369407490897
Epoch 11 / 20, Loss: 1.7683418646731288
Epoch 12 / 20, Loss: 1.7560087997640315
Epoch 13 / 20, Loss: 1.7448427204283399
Epoch 14 / 20, Loss: 1.7345941515969527
Epoch 15 / 20, Loss: 1.7251037038142911
Epoch 16 / 20, Loss: 1.716438345625396
Epoch 17 / 20, Loss: 1.7080758406577474
Epoch 18 / 20, Loss: 1.7002481696549243
Epoch 19 / 20, Loss: 1.6929249335616656
Epoch 20 / 20, Loss: 1.6859852562939137
